### Submodular Gradient Ascent

In [1]:
from comblearn.nn import DSF
import torch

In [2]:
device = 'cuda'

In [6]:
n = 10
mo = 50
dsf = DSF(n, 1, mo, [3, 4, 3], 20).to(device)

In [7]:
x = torch.tensor([1, 0, 1, 0, 0, 0, 0, 0, 0, 1]).float().to(device)
dsf(x)

tensor([38.9064], device='cuda:0', grad_fn=<MinimumBackward0>)

In [8]:
list(dsf.parameters())[0]

Parameter containing:
tensor([[0.7051, 0.1668, 0.1185],
        [1.4048, 0.6906, 0.9834],
        [1.1202, 0.1517, 1.5553],
        [0.2875, 0.4174, 1.1005],
        [1.2695, 0.1578, 1.7497],
        [1.6367, 0.4558, 0.4135],
        [0.8231, 0.3081, 0.4602],
        [1.0114, 0.3765, 0.2798],
        [0.7217, 0.5841, 0.3563],
        [1.2543, 1.6676, 0.3427]], device='cuda:0', requires_grad=True)

In [9]:
from torch import nn

class DSFWrapper(nn.Module):
    def __init__(self, n, dsf):
        super().__init__()
        self.weights = nn.Parameter(torch.zeros((n, ))).float()
        self.submodular = dsf

    def forward(self, x):
        mask = (x > 0).float()
        return self.submodular(mask * self.weights)

In [10]:
from torch.optim import SGD

eps = 0.05
T = int((n / eps) ** 2)
lr = 2e-5

wrapper = DSFWrapper(n, dsf).to(device)

In [11]:
list(wrapper.parameters())

[Parameter containing:
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0',
        requires_grad=True),
 Parameter containing:
 tensor([[0.7051, 0.1668, 0.1185],
         [1.4048, 0.6906, 0.9834],
         [1.1202, 0.1517, 1.5553],
         [0.2875, 0.4174, 1.1005],
         [1.2695, 0.1578, 1.7497],
         [1.6367, 0.4558, 0.4135],
         [0.8231, 0.3081, 0.4602],
         [1.0114, 0.3765, 0.2798],
         [0.7217, 0.5841, 0.3563],
         [1.2543, 1.6676, 0.3427]], device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor([0., 0., 0.], device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor([[1.6228, 0.1074, 1.0731, 0.0495],
         [1.0709, 0.6211, 0.8073, 0.2202],
         [0.6275, 1.2969, 1.8320, 0.6924]], device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor([0., 0., 0., 0.], device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor([[0.2090, 0.4987, 0.7444],
         [0.7260, 0.8825, 0.1297],
         [0.217

In [12]:
batch_size = 10
last_pred = -1.0

for i in range(T // batch_size):
    print(f"Step {i}/{T // batch_size}")
    
    m = torch.ones((batch_size, n)).float().to(device)
    pred = wrapper(m).mean()
    print(pred)
    pred.backward()
    g = wrapper.weights.grad
    
    if pred == last_pred:
        break
    else:
        last_pred = pred

    print('G:', g)
    with torch.no_grad():
        wrapper.weights.add_(lr * g)
        wrapper.weights.abs_()


    print('W:', list(wrapper.parameters())[0])
    # print('P:', list(dsf.parameters())[0][0])
    
# dsf.requires_grad_(True)

Step 0/4000
tensor(0., device='cuda:0', grad_fn=<MeanBackward0>)
G: tensor([ 4.4753, 17.2685, 18.6722, 12.8355, 20.9790, 11.9452,  8.6241,  8.0556,
         8.6614, 15.7590], device='cuda:0')
W: Parameter containing:
tensor([8.9505e-05, 3.4537e-04, 3.7344e-04, 2.5671e-04, 4.1958e-04, 2.3890e-04,
        1.7248e-04, 1.6111e-04, 1.7323e-04, 3.1518e-04], device='cuda:0',
       requires_grad=True)
Step 1/4000
tensor(0.0375, device='cuda:0', grad_fn=<MeanBackward0>)
G: tensor([ 8.9505, 34.5371, 37.3443, 25.6709, 41.9579, 23.8903, 17.2482, 16.1112,
        17.3227, 31.5179], device='cuda:0')
W: Parameter containing:
tensor([0.0003, 0.0010, 0.0011, 0.0008, 0.0013, 0.0007, 0.0005, 0.0005, 0.0005,
        0.0009], device='cuda:0', requires_grad=True)
Step 2/4000
tensor(0.1126, device='cuda:0', grad_fn=<MeanBackward0>)
G: tensor([13.4258, 51.8056, 56.0165, 38.5064, 62.9369, 35.8354, 25.8723, 24.1667,
        25.9841, 47.2769], device='cuda:0')
W: Parameter containing:
tensor([0.0005, 0.0021, 0.

In [19]:
wrapper.weights.clone().detach()

tensor([0.1280, 0.4939, 0.5340, 0.3671, 0.6000, 0.3416, 0.2466, 0.2304, 0.2477,
        0.4507], device='cuda:0')

In [20]:
y = torch.zeros((3, 4)).to(device)
y

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]], device='cuda:0')

In [22]:
y[:, 2] = torch.tensor([1, 2, 3])
y

tensor([[0., 0., 1., 0.],
        [0., 0., 2., 0.],
        [0., 0., 3., 0.]], device='cuda:0')

In [23]:
eval("torch.optim.SGD")

torch.optim.sgd.SGD

In [39]:
import yaml
import os

print(os.getcwd())

with open("../config/config_rg.yaml") as fp:
    cfg = yaml.load(fp, Loader=yaml.FullLoader)

/home/mohammad_hosseini/auction/notebooks


In [40]:
cfg

{'auction': {'bidders': {'list': [0, 1, 2, 3, 4],
   'value-function': [{'cls': 'comblearn.data.DSFValueFunction',
     'max-out': 100,
     'hidden-sizes': [2, 3, 2],
     'alpha': 500}]},
  'items': [0, 1, 2, 3, 4, 5, 6, 7],
  'q-init': 500,
  'q-max': 510,
  'device': 'cuda',
  'marginal': False},
 'dsf-learner': {'optimizer': 'torch.optim.SGD',
  'learning-rate': 0.001,
  'epochs': 1000,
  'device': 'cuda'}}

In [35]:
from comblearn.data import DSFValueFunction

eval('DSFValueFunction(items, 100, [2, 3, 2], 500)')

NameError: name 'items' is not defined

In [41]:
from tensordict.tensordict import TensorDict

In [43]:
t = TensorDict({'A': torch.tensor([1, 2, 3]), 'B': torch.tensor([4, 5, 6])}, batch_size=1)

RuntimeError: batch dimension mismatch, got self.batch_size=torch.Size([1]) and value.shape[:self.batch_dims]=torch.Size([3]) with value tensor([1, 2, 3])

In [44]:
{b: i for b, i in zip(range(3), range(3))}

{0: 0, 1: 1, 2: 2}